# [MICRO-452:][MICRO-452] Project Report - Groupe XX



<p><b>Authors:</b>  Stephen Monnet, David Rüegg, Julien Burkhard, Sylvain Jacquart<br>
<b>Supervisors:</b> Prof. Francesco Mondada, Laila El-Hamamsy<br>
<b>Due date:</b>    December 12th, 2021</p>

[MICRO-452]: https://moodle.epfl.ch/pluginfile.php/2727652/mod_resource/content/3/Week%209%20-%20Project%20Presentation.pdf

<h1><center> Thyminotaur - εσκαπινγ θε μαζε (Εscaping the maze) </center></h1>
<img src="images/angry_thymio.gif" style="width: 600px;">

<h1>Table of Contents<span class="tocSkip"></span></h1><br>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1.&nbsp;&nbsp;</span>Introduction</a></span></li></ul><ul class="toc-item"><li><a href="#Project-summary-,-hardware-and-choices" data-toc-modified-id="Project-summary-,-hardware-and-choices-2"><span class="toc-item-num">2.&nbsp;&nbsp;</span>Project summary, hardware and choices</a></li></ul><ul class="toc-item"><li><span><a href="#Vision" data-toc-modified-id="Vision-3"><span class="toc-item-num">3.&nbsp;&nbsp;</span>Vision</a></span></li></ul><ul class="toc-item"><li><a href="#Global-navigation" data-toc-modified-id="Global-navigation-4"><span class="toc-item-num">4.&nbsp;&nbsp;</span>Global navigation</a></li></ul><ul class="toc-item"><li><a href="#Local-navigation" data-toc-modified-id="Local-navigation-5"><span class="toc-item-num">5.&nbsp;&nbsp;</span>Local navigation</a></li></ul><ul class="toc-item"><li><a href="#Filtering" data-toc-modified-id="Filtering-6"><span class="toc-item-num">6.&nbsp;&nbsp;</span>Filtering</a></li></ul><ul class="toc-item"><li><a href="#Motion-control" data-toc-modified-id="Motion-control-7"><span class="toc-item-num">7.&nbsp;&nbsp;</span>Motion control</a></li></ul><ul class="toc-item"><li><a href="#Main" data-toc-modified-id="Main-8"><span class="toc-item-num">8.&nbsp;&nbsp;</span>Main</a></ul>

# I. Introduction
<a class="anchor" id="Introduction"></a>
<p style='text-align: justify;'> 
    This mobile robotics project is inspired from the old legend of <i>Theseus and the Labyrinth</i>. It all started during the SHS course <i>Myths of the ancient Mediterranean Sea</i> with the following question:</p>

**What if the Minotaur, trapped forever in the vast Labyrinth built on Creta island, could find his way out and live his best life under the greek sun?**

The major goal of this project is therefore to help the little Thyminotaur to find his way out of the maze! To help him with his quest for freedom, the 5 following tools are implemented in the project.  

1. **Vision**: Placed above the maze, a camera detects the walls, maze exits and the position of the robot. The optimal path is then computed with the vision information. We also perform calibration prior to the execution of the program, to guarantee an acurate tracking of the Thymio.


2. **Global Navigation**: With the help of the camera, the global path to the nearest exit of the labyrinth is compiled and send as the route to follow for the Thymio. The nearest exit is detected with a common cost-function algorithm.


3. **Local Navigation**: With the help of the 5 front infrared sensors, the Thymio can avoid unexpected obstacles along the way. We assign a different weight to each sensor, to ensure a smooth obstacle avoidance behaviour a minimum of wall-crossing during these manoeuvers.


4. **Filtering**: An extended Kalman Filter is implemented in this project, as a support to the vision part. Using this filter, we are able to track the robot with a precision in linear speed/position of XX and in angular speed/angle of YY. <font color='red'>TO COMPLETE</font>


5. **Motion Control**: Once the path is computed, the information is transmitted to the Thymio which starts moving.


The main folder of the project can be found following the repository: https://github.com/jbyuki/labyrinth-project

The global decision tree for the project can be summarized in the figure below. Our Thyminotaur is robust to a camera blinding (thanks to the Kalman filter) but not to kidnapping; after all, it is hard to kidnap people surrounded by labyrinth walls...

<img src="images/Decision_tree.PNG" style="width: 600px;">

# II. Project summary, hardware and choices
<a class="anchor" id="Project-summary-,-hardware-and-choices"></a>
### Project summary

<img src="images/GlobalExplenation.PNG" style="width: 600px;">

1. We place the Thymio on a random cell of the labyrinth. The camera detects the *ArUco Marker* on the Thymio, compiling its position and orientation. 


2. The labyrinth is drawn with 2 exits; using the Dijkstra’s algorithm, the Thymio follows the path to the nearest exit of the maze (green path on the image).


3. During the path following, some unexpected obstacles will block the road to our adventurous Thyminotaur. Thanks to his local avoidance abilities (based on his proximity sensors/neural network architecture combination), no unexpected event will stand between him and the exit. He bypasses every obstacle until the end of his journey.


4. The Thymio stops at the exit of the Labyrinth, satisfied with his regained freedom.

### Hardware
<p style='text-align: justify;'>
The labyrinth covers a 4x5 cells unit (17cm x 20cm each) and is printed on an A0 white paper (shout-out to the PolyRepro for their fast delivery). To ensure a good camera tracking, the Thymio carries an <i> ArUco Marker</i>, which allows to detect the position and orientation of the robot. The marker is stick to <i>Lego bricks</i> and mounted directly on top of the Thymio, as you can see on the picture below.</p>

<img src="images/combined_pic_thymio_lab.PNG" style="width: 600px; ">


### Relevant choices made for this project
1. At first, a solid labyrinth made of wood was considered enclosing the Thymio. After several tries and prototypes, the full paper option (black lines printed on a large white sheet) was selected for its practicality and easiness of implementation with the vision part.

    
2. To mark out the available space of the Thymio (his playground), 4 additional arUco Markers are placed on each corner of the sheet. The camera placed above the labyrinth is never perfectly aligned with the sheet, both in position and angle; the arUcodes, combined with extrem-positions calibration, allow a straightforward flattening of the image at the vision initialization.

# III. Vision
<a class="anchor" id="Vision"></a>

In the Thyminotaur project, the whole vision data is provided by a single static camera located over the robot's scope of operation. We use vision during the initialization phase (labyrinth recognition, full-scale calibration, corners detection) and while the robot is moving (Thymio detection). We coded the entire vision part using the Python library <i>Open Source Computer Vision</i>, or [open cv][cv2] in short.

We assigned arUco Markers to every critical variable of the project (the moving Thymio and the 4 corners) to ensure consistency in the code. These markers are a simple yet efficient way to convert information from a full vision scope to a code of variables and raw data (images on left & center). For corners, it is just a way to fix the operating frame of the robot. On the Thymio, the marker allows us to track the position and angle of the Thyminotaur from the starting point in real time.

The labyrinth is made of large, 20mm-width black lines printed of white paper to achieve the best contrast. This simplifies our program in defining the possible paths for the robot (image on right). Since system vision is a key element in both the global navigation algorithm and for general system motion control, we directly included the vision functions in each class rather than putting them in their own separate class. <br>

<img src="images/vision_3_image.PNG" style="width: 900px;">
    
The `detect_labyrinth` function below summarizes how black lines, on a A0 paper sheet, can be recognized as a concrete object. This is just once relevant function of the vision part in this project; you can find more on the `vision_utilis.py` file. The `dilate` function is an excellent way to detect all possible paths in the labyrinth, centered in the middle of corridors.
    
[cv2]: https://github.com/opencv/opencv

In [ ]:
# ----
# Gives a binary grid representation of the labyrinth (img must be grayscale)
# ----
 
def detect_labyrinth(img):
  h,w = img.shape

  # Remove AruCo
  detected = detect_aruco(img)
  erase_aruco(img, detected)


  # Threshold definition
  th = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,31,2)
  
  cv.imwrite("global_trajectory_real_resized_th.png", th)

  # Remove noise
  kernel = np.ones((10,10),np.uint8)
  kernel2 = np.ones((25,25),np.uint8)
  th = cv.morphologyEx(th, cv.MORPH_OPEN, kernel)
  # th = cv.dilate(th, kernel2, iterations=1)

  # Detect connected components
  num_labels, labels, stats, _ = cv.connectedComponentsWithStats(th, 8, cv.CV_32S)

  # Pick wall components
  result = np.zeros_like(th)
  for i in range(1, num_labels):
    if stats[i, cv.CC_STAT_AREA] > WALL_THRESHOLD:
      result[labels == i] = 255

  # temp code
  # result = th
  cv.imwrite("global_trajectory_real_resized_test.png", th)

  # Dilate along both x and y direction
  kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(20, 20))
  result = cv.dilate(result,kernel,iterations = 1)
  result = dilate_walls_max(result, [5, 5], [10, 10], [20, 20])

  result = dilate_walls_max(result, [5, 5],[10, 0], [20, 20])
  result = dilate_walls_max(result, [5, 5], [0, 10], [20, 20])

  return result

# IV. Global navigation
<a class="anchor" id="Global-navigation"></a>
Our global navigation implementation is based on the motion cost approach, as if some moves (turning, moving in diagonal) were more costly than others (moving straight).

The path to follow is made of a succession of absolute points on the XY Cartesian plane. This is the most convenient way to navigate our Thymio inside the rectangle, straight design maze. You can see an example of the camera / live implementation below.

<img src="images/Path_planning.PNG" style="width: 900px;">

# V. Local Navigation
<a class="anchor" id="Local-navigation"></a>
The local navigation runs continually while the Thymio is moving. This section is only implemented to avoid unexpected obstacles along the way, not to reorientate the Thymio after such an avoidance. Therefore, both motors are controlled by the `obstacle_avoidance_full` routine below, over the global navigation algorithm. 

This function is straightforward. It consists of assigning different weights (`diffDelta`) to each of the 5 IR proximity sensors on Thymio's front. These weights give stronger importance to external sensors over the middle one.  Using a variable threshold (which can be tuned to match environment conditions), the Thymio is capable of detecting if an unexpected obstacle stands on his way to the exit:
- When no obstacle is in sight, the robot is monitored by the global path navigation and goal tracking algorithm.
- When something's on the way, the diffDelta variable increase the speed of the motor close to the obstacle side. The closer the obstacle, the faster the motor goes.


<img src="images/gif_thymio_avoidance.gif" style="width: 400px;">

In [2]:
def obstacle_avoidance_full(node, variables):
    # global motor_left, motor_right
    try:   
        speed0 = 100       # nominal speed
        speedGain = 5      # gain used with ground gradient
        obstacleGain = 2   # gain used to avoid obstacle
        obstThr = 20       # obstacle threshold to switch state 1->0
        state = 0          # 0=gradient, 1=obstacle avoidance
        prox = variables["prox.horizontal"]

        # difference between sensors
        diffDelta = prox[0]+0.5*prox[1]-0.25*prox[2]-0.5*prox[3]-prox[4]

        if state == 0 and (prox[0] > obstThr or prox[1] > obstThr or prox[2] > obstThr or prox[3] > obstThr or prox[4] > obstThr):
        # switch from goal tracking to obst avoidance if obstacle detected
           state = 1

        elif state == 1 and (prox[0] < obstThr and prox[1] < obstThr and prox[2] < obstThr and prox[3] < obstThr and prox[4] < obstThr):
        # switch from obst avoidance to goal tracking if obstacle got unseen
             state = 0

             if state == 0 :
             # goal tracking: turn toward the goal
                motor_left = speed0 - speedGain * diffDelta
                motor_right = speed0 + speedGain * diffDelta           
                
             else :
             # obstacle avoidance: accelerate wheel near obstacle
                motor_left = speed0 + obstacleGain* diffDelta
                motor_right = speed0 + obstacleGain* (-diffDelta)

        node.send_set_variables(motors(motor_left, motor_right))
    except KeyError:
        pass  # prox.horizontal not found

# VI. Filtering
<a class="anchor" id="Filtering"></a>
We choose to tackle the filtering part of this project by implementing an Extended Kalman Filter.


7.8/21
estimer le probabilité
distri de proba, à ajuster en continu. Quand il avance dans l'ombre (sans cam), plus la gaussienne s'applatit.
Camera = sensor absolu, pas d'incertitude sur la position / sans cam = accéléromètre, vitesse -> sensor relatif
Kalmann ->, général, Gaussien

# VII. Motion Control
<a class="anchor" id="Motion-control"></a>

The motion control aims to provide the correct motors command to the robot, depending on its position ($P_{x_r}$, $P_{y_r}$, $\alpha_r$) and the path to follow computed previously. The control approach is directly based on the fact that the trajectories can only be composed of straight lines in the actual maze. Therefore the robot can accurately follow the path simply by moving to the intersection points of the straight lines generated by the Dijkstra’s algorithm.
The motors command can be decomposed into three terms :

$$\vec v_{mot} = \vec v_{orientation} + \vec v_{position} + \vec v_{obstacle}$$

1. **Orientation**: When the next point $P_{k+1}$ where the robot needs to go is known, the robot has to correct its orientation $\alpha_r$. A simple P regulator is used to miminimze the error $\alpha_e = \alpha_r - \alpha_c$, where $\alpha_c$ is the commanded orientation of the robot. The following equations show how the angular command is computed and then how is deduced the motors speed (orientation part).

$$\alpha_c = - atan2(\frac{Py_{k+1} - P_{y_r}}{Px_{k+1} - P_{x_r}})$$

$$\vec v_{orientation} = \pm K_{p_\alpha} \cdot \alpha_e$$
    
  The $\pm$ sign is used to notify that the robot needs to turn, so the speed of the left and the right wheel need to be in  the opposite directions.


2. **Position**: Now that the robot has the right orientation, it needs to move forward to reach the point $P_{k+1}$. The motor command for this part is not computed using a classic P control law because it is based on the angular error. The objective is to have a constant speed when the robot is travelling between two points with the right orientation but also to have a speed close to zero (for this part) when the orientation is wrong : 

$$\vec v_{position} = K_{p_d} \cdot (180 - \mid\alpha_e\mid)$$

<img src="images/MotionControl_scheme_net.png" style="width: 500px;">

3. **Obstacle avoidance**: This part also set a speed command using rules that are already described in its own section.

These three parts are the main ideas of the motion control approach used for this project.

## Implementation problematics

This motion control implementation involved being particularly careful about the following points :
   - **Referentials consideration** : Two angles need to be considered for this problem, the orientation of the robot $\alpha_r$ and the angle between the vector $\vec d(t)$ and the $\vec y$ axis, which is used to set the orientation command of the robot. This was more difficult than expected to ensure consistence between these two values in any situation.
   - **Angular discontinuity** : Because an angle is defined between $0 ~rad$ and $2\pi ~rad$ it creates a hudge discontinuity when the robot angle is close to $2\pi ~rad$ and suddenly goes to $0 ~rad$. For example, suppose the robot needs to reach an orientation of $0 rad$ (which is the discontinuity location) and has a current orientation of $\pi/6 ~rad$, then the error is $\alpha_e = \pi/6 - 0 = 0 ~rad$. The generated motors command will reduce the error to reach $\alpha_e = 0 ~ rad$ which means that $\alpha_r = 0 ~rad$. But if the robot goes a bit further (overshoot) then its orientation will jump from, approximately 0 rad, to approximately $2\pi ~rad$. Therefore the error becomes $\alpha_e = 2\pi - 0 = 2\pi ~rad$ and so the robot will continue to turn on itself. This effect has been avoided by limiting the orientation error $\alpha_e$ between $-\pi$ and $\pi$. For example if the computed error is $3\pi/2~rad$ then it is modified to be equal to $-\pi/2~rad$ which is equivalent.
    

## Improvement of the motion control

After several trials, the following problems have been identified :
   - **Overshoot after reaching a point** : Because of the command law presented previously, the speed of the robot doesn't decrease when it arrives close to its point to reach ($P_{k+1}$. So in this configuration, there is a trade-off between the speed of the robot between two points and the overshoot. To solve this problem, the control law has been improved by modifying the gains $K_{P_\alpha}$ and $K_{P_d}$ depending if the robot is close to the point to reach or not. If the robot is close (distance <= 20 pixels) to $P_{k+1}$ the gains are $K_{P_d} = 0.5$ and $K_{P_\alpha} = 4$. If the robot is not close (distance > 20 pixels) to $P_{k+1}$ then the gains are $K_{P_d} = 1$ and $K_{P_\alpha} = 1$. In this configuration, the robot can easily turn without overshoot when it is close to $P_{k+1}$, because the weight of the orientation command increase relatively to the weight of the position command. It can also move faster when it is far from $P_{k+1}$.


   
   - **Conflict between $\vec v_{position}$ and $\vec v_{orientation}$** : This problem is similar to previous one. It first appeared when the robot started its run very close (approximately 10 pixels) to the next point to reach $P_{k+1}$, oriented in the opposite direction of this latter. In this configuration, the robot was not able to reach the point beceause the linear speed command was too high compared to the rotation command speed. In fact, the robot was only able to turn around the point. This problem has also been corrected thanks to the previously shown modification of the control law, which gives more weight to the orientation command when the robot is close (distance < 20 pixels) to $P_{k+1}$.
    

# VIII . Main
<a class="anchor" id="Main"></a>
If you desire to execute the code from the notebook, the following cells must be run.
1. Cloning the github repository
2. Modifying the path 
3. Running the main script

In [ ]:
#Instal the required packages, p.ex:
# !pip install opencv-python tqdm matplotlib numpy ipywidgets python-dotenv pyserial
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
!git clone https://github.com/jbyuki/labyrinth-project.git

In [ ]:
%cd labyrinth-project

In [ ]:
!python main_script.py